# 2.6 ICESat-2 biomass 30m ATL08

In [1]:
#### i) Description
#### This notebook use GEDI-derived AGB models for estimating sparse ICESat-2 AGB using 30 m ATL08 product.

#### ii) How the algorithm works?
#### GEDI-derived AGB models are loaded in R and applied over the ICESat-2 30m ATL08 data. 

#### iii) Inputs
####  - gedi_model: GEDI-derived AGB model
####  - ice2_30_atl08: list containing the path to the ATL08 files
####  - outdir: output directory for saving ICESat-2 AGB estimates
####  - offset: offset applied in the model
####  - gedi_vars: GEDI variables used in the model
####  - ice_vars: ICESat-2 variables equivalent to GEDI variables
####  - overwrite: if TRUE will overwrite existing output files

#### iii) Outputs
####  -ICEsat-2 AGB estimates are saved as csv in the outdir


#----------------------------------------------#
############# functions ########################
#----------------------------------------------#
applyToBoreal<-function(model,ice2_30_atl08, outdir=getwd(), offset=10, gedi_vars=c('RH_98', 'RH_20'), ice_vars=c("h_can", "rh25"), overwrite=FALSE){
  
  pb <- txtProgressBar(min = 0, max = length(ice2_30_atl08), style = 3)
  i.s=0
  
  for ( i in ice2_30_atl08){
      print(i)
    i.s<-i.s+1
    setTxtProgressBar(pb, i.s)
       # reading ICEsat-2 30m ATL08 file
    L <- readLines(i, n = 1)
        
    if (!length(L)==0) {
            
    if (grepl(";", L)) ice_i<-read.csv2(i, head=T) else ice_i<-read.table(i, sep=",", head=T)
      
    #print(head(ice_i))
    ice_i<-as.data.frame(ice_i)
    ice_i_sub<-na.omit(sapply(ice_i[,ice_vars],function(x) as.numeric(as.character(x))))
    
    list_outputs<-list.files(outdir)
    if (!is.null(nrow(ice_i_sub))) {
      
      if (file.exists(file.path(outdir, basename(i)))) {
        if (overwrite==FALSE){
        print(paste0("Skipping file",basename(i),". It has been already processed"))
        include=FALSE  
        } else {
          include=TRUE  
         } 
      } else { include=TRUE}
      
      if ( include==TRUE){
      
      newdata <- as.data.frame(sqrt(ice_i_sub))
      
      #rename variables to match model
      names(newdata)=gedi_vars
      
      #adjust for offset in model fits (note, this was 20 for ages, and now it's 100; essentially we added to all the height metrics so they would never be negative)
      newdata <- newdata+offset
      
      #define C
      C <- mean(model$fitted.values)/mean(model$model$`sqrt(AGBD)`)
      
      #we multiply by C in case there is a systematic over or under estimation in the model (bias correction)
      ice_i$agbd_pred<-C*(predict(model, newdata=newdata)^2)
      write.table(ice_i,paste0(outdir, '/', basename(i)), sep=";", row.names = F)
      }
     }  
    }
   }                           
  close(pb)
}

In [2]:
# Importing GEDI-derived AGB models
gedi_model <- readRDS('~/csilva/gedi_agb_model/predict_ENT_coarse_fit_ENT_sqrt_sqrt_Snowdon_1.rds')
#this is a simple placeholder of a square root transformed model
#read in some icesat2 data, then make a dataframe of sqrt(RH98) and sqrt(RH20)
#When applying this automatically we can search the variable names in the model object for application, but for now we can hard code

In [3]:
# summary
summary(gedi_model)


Call:
lm(formula = paste("sqrt(AGBD)~", paste(paste("sqrt(", predictor_names, 
    ")", sep = ""), collapse = "+")), data = train_data_fit_stratum)

Residuals:
     Min       1Q   Median       3Q      Max 
-13.7057  -2.1572  -0.0334   2.0397  12.7109 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) -138.2682     3.4818  -39.71   <2e-16 ***
sqrt(RH_20)    5.8403     0.3922   14.89   <2e-16 ***
sqrt(RH_98)    8.2150     0.1837   44.72   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 3.48 on 2002 degrees of freedom
Multiple R-squared:  0.6505,	Adjusted R-squared:  0.6501 
F-statistic:  1863 on 2 and 2002 DF,  p-value: < 2.2e-16


In [4]:
# GEDI and ICEsat-2 variables
coef(gedi_model)
gedi_vars<-c('RH_20', 'RH_98')
ice_vars<-c('rh25', 'rh95')

# importing list of ICESat-2 30m ATL08
icews<-"~/r2d2/output_23/" # using 100m ATL08 for now
ice2_30_atl08<-list.files(icews, ".csv", full.names = T)

# print firt file in the list
ice2_30_atl08[1]
# length of the list
length(ice2_30_atl08)

(Intercept) sqrt(RH_20) sqrt(RH_98) 
-138.268244    5.840301    8.214981

[1] "/projects/r2d2/output_23//ATL08_20181014001049_02350102_003_01.csv"

[1] 1283

In [5]:
# output directory
outdir='~/r2d2/output_notebook_2.6'

# adjust for offset in model fits (note, this was 20 for ages, and now it's 100; essentially we added to all the height metrics so they would never be negative)
offset=10

In [6]:
# Apply GEDI AGB model to boreal
applyToBoreal(model=gedi_model,ice2_30_atl08=ice2_30_atl08, outdir=outdir, offset=offset, gedi_vars=gedi_vars, ice_vars=ice_vars, overwrite=FALSE)

  |                                                                      |   0%[1] "/projects/r2d2/output_23//ATL08_20181014001049_02350102_003_01.csv"
[1] "Skipping fileATL08_20181014001049_02350102_003_01.csv. It has been already processed"
[1] "/projects/r2d2/output_23//ATL08_20181014001920_02350103_003_01.csv"
[1] "Skipping fileATL08_20181014001920_02350103_003_01.csv. It has been already processed"
[1] "/projects/r2d2/output_23//ATL08_20181014014507_02360102_003_01.csv"
[1] "Skipping fileATL08_20181014014507_02360102_003_01.csv. It has been already processed"
[1] "/projects/r2d2/output_23//ATL08_20181014015337_02360103_003_01.csv"
[1] "Skipping fileATL08_20181014015337_02360103_003_01.csv. It has been already processed"
[1] "/projects/r2d2/output_23//ATL08_20181014082120_02400105_003_01.csv"
[1] "Skipping fileATL08_20181014082120_02400105_003_01.csv. It has been already processed"
[1] "/projects/r2d2/output_23//ATL08_20181014082645_02400106_003_01.csv"
[1] "Skipping fileATL08_2018

Warning message in readLines(i, n = 1):
“line 1 appears to contain an embedded nul”Warning message in readLines(i, n = 1):
“incomplete final line found on '/projects/r2d2/output_23//ATL08_20181120082034_08050106_003_01.csv'”Warning message in read.table(i, sep = ",", head = T):
“line 1 appears to contain embedded nulls”Warning message in read.table(i, sep = ",", head = T):
“incomplete final line found by readTableHeader on '/projects/r2d2/output_23//ATL08_20181120082034_08050106_003_01.csv'”

ERROR: Error in read.table(i, sep = ",", head = T): empty beginning of file
